<h1>Peer Graded Assignment - Segmentation and Clustering in the neighbourhoods in Toronto

<h2> A] Scraping and cleaning the data from Wikipedia

<h3>1) Reading Toronto data which was extracted into a csv file from Wikipedia

In [194]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
'''
Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. 
Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
'''

"\nAnother way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. \nHere is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/\n"

<h2> B] Dataframe after adding the latitudes and longitudes

<h3>1) Reading the coordinates file

In [195]:
cord_path = '/Users/Rohit/Desktop/Capstone_projs/India_Data/pune_coordinates-2.csv'
df_cord = pd.read_csv(cord_path)
df_cord.head()

,Area Name,Postal Code,Latitude,Longitude
0,Adivare,410509,19.159047,73.696210
1,AFMC,411040,18.506342,73.890503
2,Agoti,413132,18.226457,74.975982
3,Airport,411032,18.578542,73.908641
4,Akurdi,411035,18.646468,73.774262


In [196]:
df_cord = df_cord.append(pd.DataFrame({"Area Name":['Koregaon Park', 'Kalyani Nagar', 'Boat Club Road', 'Sopan Baug', 'Undri', 'Fatima Nagar'],
              "Postal Code":['411001', '411006', '411001', '411001','411060','411022'],
              "Latitude":[18.536207,18.546328,18.539517,18.5142,18.4567,18.5042 ],
              "Longitude":[73.893974,73.903313,73.878978,73.9040,73.9095,73.9014]}), ignore_index=True)
#df2 = pd.DataFrame({"a":[1, 2, 3], 
#                    "b":[5, 6, 7]}) 
df_cord.head()

,Area Name,Postal Code,Latitude,Longitude
0,Adivare,410509,19.159047,73.696210
1,AFMC,411040,18.506342,73.890503
2,Agoti,413132,18.226457,74.975982
3,Airport,411032,18.578542,73.908641
4,Akurdi,411035,18.646468,73.774262


In [197]:
df_cord.sort_values(['Area Name'], axis=0, inplace=True)
df_cord.head()

,Area Name,Postal Code,Latitude,Longitude
1,AFMC,411040,18.506342,73.890503
0,Adivare,410509,19.159047,73.696210
2,Agoti,413132,18.226457,74.975982
3,Airport,411032,18.578542,73.908641
4,Akurdi,411035,18.646468,73.774262


<h3>2) Merging coordinates file with the dataframe

In [198]:
df_cord['Postal Code'] = df_cord['Postal Code'].astype(str)
Pune_filtered = df_cord[df_cord['Postal Code'].str.startswith('411')]
Pune_filtered.rename(columns = {'Area Name':'Neighborhood'}, inplace = True) 
   
#merge Pune_grouped with Pune_data to add latitude/longitude for each neighborhood
#Pune_merged = Pune_merged.join(df.set_index('Postal Code'), on='Postal Code')
#Pune_merged = Pune_merged[['Postal Code','Borough','Neighborhood','Latitude', 'Longitude']]
Pune_filtered.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Neighborhood,Postal Code,Latitude,Longitude
1,AFMC,411040,18.506342,73.890503
3,Airport,411032,18.578542,73.908641
4,Akurdi,411035,18.646468,73.774262
16,Ambegaon BK,411046,18.402869,73.853668
24,Ammunition Factory Khadki,411003,18.567988,73.857668


<h2> C] Exploring and Clustering Datasets

<b>Defining credentials for Foursquare

In [205]:
#Rohit
#CLIENT_ID = 'ZSJT2FU5RCNHDXX1M015PKIGD2CG3N3DQ4VHTA30FLDYHWKK' # your Foursquare ID
#CLIENT_SECRET = 'BDLAW0PKZ3AG3ZRMJCLYW2Q0M2ZMZBSYBLDAPWQVBY1FVGFE' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
#LIMIT = 300
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#Divya
CLIENT_ID = 'A2V2SIQQ2T1ZI5EMEYMZD5GER4YTOMXLVSEVZ4NBXYWSZ1S2' # Foursquare ID
CLIENT_SECRET = 'O5YVJFZ2Q4H0R50FOLWXSV4JCVPWNYAKQUKPRH4S4V00SZQN' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 250
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A2V2SIQQ2T1ZI5EMEYMZD5GER4YTOMXLVSEVZ4NBXYWSZ1S2
CLIENT_SECRET:O5YVJFZ2Q4H0R50FOLWXSV4JCVPWNYAKQUKPRH4S4V00SZQN


<h3> 2) Define a function for getting json file and converting it to a dataframe with venues

In [206]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        results = results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3> 3) Feed in Toronto filtered dataset in the function

In [209]:
Pune_venues = getNearbyVenues(names=Pune_filtered['Neighborhood'],
                                   latitudes=Pune_filtered['Latitude'],
                                   longitudes=Pune_filtered['Longitude'],
                                   radius = 300
                                  )

AFMC          
Airport          
Akurdi          
Ambegaon BK        
Ammunition Factory Khadki      
Anandnagar          
Armament          
Aundh Camp        
Aundh T.S.        
Aundh          
Bajirao Road        
Baner Gaon        
Baner Road        
Bavdhan          
Bhavani Peth        
Bhosari I.E.        
Bhosarigoan          
Bhusari Colony        
Bibvewadi          
Boat Club Road
Botanical Garden        
C D A      
C M E      
Chikhali          
Chikhlee          
Chinchwad East        
Chinchwadgaon          
Congress House Road      
Dapodi Bazar        
Dapodi          
Deccan Gymkhana        
Dhankawadi          
Dhanori          
Dhayari          
Dighi Camp        
Donje          
Dr.B.A. Chowk        
Dukirkline          
East Khadki        
Ex. Serviceman Colony      
Fatima Nagar
Film Institute        
Ganeshkhind          
Ghorpade Peth        
Ghorpuri Bazar        
Gokhalenagar          
Gondhale Nagar        
Govt. Polytechnic        
Guruwar Peth        
H.E.

In [212]:
Pune_venues.head()
Pune_venues.to_csv('Pune_ven.csv')

<h3> 4) Checking size of the resulting dataframe

In [215]:
Bakery = Pune_venues[Pune_venues['Venue Category'].str.startswith('Bakery')]
print(Bakery.shape)
Bakery

(16, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,Airport,18.578542,73.908641,Baker Street,18.579810,73.908800,Bakery
21,Aundh,18.555964,73.804439,La Bouchee d'Or,18.558395,73.803787,Bakery
70,Boat Club Road,18.539517,73.878978,La Bouchee d'Or,18.538931,73.876677,Bakery
85,Congress House Road,18.523303,73.853551,Ramsar Bakery,18.524994,73.853729,Bakery
113,Dr.B.A. Chowk,18.523491,73.869549,MURTHY'S,18.521955,73.869659,Bakery
152,Gokhalenagar,18.530733,73.830319,Pune Baking Company,18.531972,73.829834,Bakery
237,Khondhwa KH,18.513021,73.878765,Kayani Bakery,18.514797,73.879860,Bakery
283,Market Yard,18.485306,73.874227,Unity Bakers,18.485730,73.871915,Bakery
297,N I B M,18.476615,73.896839,Bakers Point,18.477303,73.895935,Bakery
314,Navsahyadri,18.482529,73.809557,Honeywell,18.482834,73.808159,Bakery


In [214]:
# create map for bakeries
latitude = 18.516726
longitude = 73.856255
map_bak = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, ven in zip(Bakery['Venue Latitude'], Bakery['Venue Longitude'], Bakery['Neighborhood'], Bakery['Venue']):
    label = folium.Popup(str(ven) + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='b',
        fill=True,
        fill_color='r',
        fill_opacity=0.9).add_to(map_bak)
       
map_bak

<h3>Zone wise data </h3>

<h4>filter bakery data </h4>

<h3> 6) Finding out how many unique categories can be curated from all the returned venues

In [263]:
print('There are {} uniques categories.'.format(len(Pune_Venues_Zones['Venue Category'].unique())))

KeyError: 'Venue Category'

<h3>7) Using one hot encoding for Venues

In [20]:
# one hot encoding
Pune_onehot = pd.get_dummies(Pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Pune_onehot['Neighborhood'] = Pune_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Pune_onehot.columns[-1]] + list(Pune_onehot.columns[:-1])
Pune_onehot = Pune_onehot[fixed_columns]

Pune_onehot.head()

,Neighborhood,ATM,Airport Lounge,Airport Service,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Mattress Store,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Resort,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Watch Shop,Women's Store,Zoo
0,AFMC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AFMC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,AFMC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AFMC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Airport,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
Pune_onehot.shape

(1047, 141)

<h3>8) Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
Pune_grouped = Pune_onehot.groupby('Neighborhood').mean().reset_index()
Pune_grouped

,Neighborhood,ATM,Airport Lounge,Airport Service,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hockey Field,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Mattress Store,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Resort,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Watch Shop,Women's Store,Zoo
0,AFMC,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Airport,0.000000,0.2,0.2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.00000,0.

In [23]:
Pune_grouped.shape

(117, 141)

<h3>9) Let's print each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5

for hood in Pune_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Pune_grouped[Pune_grouped['Neighborhood'] == hood].T.reset_index()
    
    #print(temp)
    temp.columns = ['venue','freq']
    #print(temp)
    temp = temp.iloc[1:]
    
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AFMC          ----
                venue  freq
0           Racetrack  0.25
1  Chinese Restaurant  0.25
2        Cupcake Shop  0.25
3       Track Stadium  0.25
4    Parsi Restaurant  0.00


----Airport          ----
                 venue  freq
0      Airport Service   0.2
1       Airport Lounge   0.2
2  Fried Chicken Joint   0.2
3        Deli / Bodega   0.2
4           Smoke Shop   0.2


----Akurdi          ----
                  venue  freq
0                   Gym  0.25
1        Ice Cream Shop  0.12
2  Fast Food Restaurant  0.12
3           Coffee Shop  0.12
4                Bakery  0.12


----Ambegaon BK        ----
              venue  freq
0  Business Service   1.0
1               ATM   0.0
2  Parsi Restaurant   0.0
3       Music Venue   0.0
4         Nightclub   0.0


----Ammunition Factory Khadki      ----
            venue  freq
0             ATM   0.5
1  Ice Cream Shop   0.5
2   Women's Store   0.0
3            Park   0.0
4        Mountain   0.0


----Anandnagar          --

                  venue  freq
0                 Hotel  0.29
1  Fast Food Restaurant  0.24
2           Chaat Place  0.06
3            Food Truck  0.06
4     Indian Restaurant  0.06


----Kapad Ganj        ----
                     venue  freq
0  Fruit & Vegetable Store  0.25
1           Ice Cream Shop  0.25
2       Miscellaneous Shop  0.25
3              Snack Place  0.25
4                     Park  0.00


----Karvenagar          ----
                           venue  freq
0              Indian Restaurant  0.21
1                 Ice Cream Shop  0.14
2  Vegetarian / Vegan Restaurant  0.14
3           Fast Food Restaurant  0.14
4                    Pizza Place  0.07


----Kasarwadi          ----
               venue  freq
0  Indian Restaurant  0.38
1   Asian Restaurant  0.12
2       Cupcake Shop  0.12
3                Bar  0.12
4             Bakery  0.12


----Kasba Peth        ----
                   venue  freq
0             Donut Shop  0.33
1     Italian Restaurant  0.33
2            C

                venue  freq
0  Italian Restaurant  0.17
1               Diner  0.17
2       Train Station  0.17
3    Department Store  0.17
4  Chinese Restaurant  0.17


----Ranjane          ----
              venue  freq
0          Mountain   1.0
1  Parsi Restaurant   0.0
2         Multiplex   0.0
3       Music Venue   0.0
4         Nightclub   0.0


----Rasta Peth        ----
                     venue  freq
0        Indian Restaurant  0.18
1     Fast Food Restaurant  0.18
2  Comfort Food Restaurant  0.09
3           Ice Cream Shop  0.09
4       Chinese Restaurant  0.09


----Raviwar Peth        ----
               venue  freq
0  Food & Drink Shop   1.0
1                ATM   0.0
2   Parsi Restaurant   0.0
3        Music Venue   0.0
4          Nightclub   0.0


----Rupeenagar          ----
               venue  freq
0  Mobile Phone Shop   1.0
1                ATM   0.0
2    Motorcycle Shop   0.0
3          Multiplex   0.0
4        Music Venue   0.0


----S.P. College        ----
    

<h3>10) Let's put that into a *pandas* dataframe

In [25]:
#Function sorted values
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Pune_grouped['Neighborhood']

for ind in np.arange(Pune_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Pune_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,AFMC,Cupcake Shop,Racetrack,Track Stadium,Chinese Restaurant,Zoo
1,Airport,Airport Lounge,Airport Service,Deli / Bodega,Smoke Shop,Fried Chicken Joint
2,Akurdi,Gym,Ice Cream Shop,Coffee Shop,Café,Fast Food Restaurant
3,Ambegaon BK,Business Service,Zoo,Fast Food Restaurant,Food Truck,Food Court
4,Ammunition Factory Khadki,Ice Cream Shop,ATM,Watch Shop,Farmers Market,Food Truck


<h3>11) Clustering Neighborhoods

In [133]:
# set number of clusters
kclusters = 5

Pune_grouped_clustering = Pune_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 4, 1, 1, 0, 1],
      dtype=int32)

<h3> 12) Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [134]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Pune_merged = Pune_filtered

# merge Pune_grouped with Pune_data to add latitude/longitude for each neighborhood
Pune_merged = Pune_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Pune_merged.head() # check the last columns!
New_Pune = Pune_merged.dropna(axis = 0, how ='any')
convert_dict = {'Cluster Labels': int, 
               } 
  
New_Pune = New_Pune.astype(convert_dict) 
New_Pune.head()

,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,AFMC,411040,18.506342,73.890503,1,Cupcake Shop,Racetrack,Track Stadium,Chinese Restaurant,Zoo
3,Airport,411032,18.578542,73.908641,1,Airport Lounge,Airport Service,Deli / Bodega,Smoke Shop,Fried Chicken Joint
4,Akurdi,411035,18.646468,73.774262,1,Gym,Ice Cream Shop,Coffee Shop,Café,Fast Food Restaurant
16,Ambegaon BK,411046,18.402869,73.853668,1,Business Service,Zoo,Fast Food Restaurant,Food Truck,Food Court
24,Ammunition Factory Khadki,411003,18.567988,73.857668,1,Ice Cream Shop,ATM,Watch Shop,Farmers Market,Food Truck


<h3>13) Finally, let's visualize the resulting clusters

In [135]:
# create map
latitude = 18.516726
longitude = 73.856255
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(New_Pune['Latitude'], New_Pune['Longitude'], New_Pune['Neighborhood'], New_Pune['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

    
for lat1, lon1, poi1, ven1 in zip(Bakery['Venue Latitude'], Bakery['Venue Longitude'], Bakery['Neighborhood'], Bakery['Venue']):
    label1 = folium.Popup(str(ven) + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label1,
        color='b',
        fill=True,
        fill_color='r',
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

<h3>14) Examining Clusters

In [136]:
New_Pune.loc[New_Pune['Cluster Labels'] == 0, New_Pune.columns[[1] + list(range(5, New_Pune.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
101,411020,Department Store,Farmers Market,Food Truck,Food Court,Food & Drink Shop
308,411003,Department Store,Bakery,Zoo,Food Truck,Food Court
307,411003,Department Store,Bakery,Breakfast Spot,Zoo,Fried Chicken Joint


In [138]:
New_Pune.loc[New_Pune['Cluster Labels'] == 1, New_Pune.columns[[1] + list(range(5, New_Pune.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,411040,Cupcake Shop,Racetrack,Track Stadium,Chinese Restaurant,Zoo
3,411032,Airport Lounge,Airport Service,Deli / Bodega,Smoke Shop,Fried Chicken Joint
4,411035,Gym,Ice Cream Shop,Coffee Shop,Café,Fast Food Restaurant
16,411046,Business Service,Zoo,Fast Food Restaurant,Food Truck,Food Court
24,411003,Ice Cream Shop,ATM,Watch Shop,Farmers Market,Food Truck
26,411051,Trail,Zoo,Farmers Market,Food Truck,Food Court
41,411007,Indian Restaurant,Ice Cream Shop,Café,Dessert Shop,Gym
47,411002,Ice Cream Shop,Dessert Shop,Vegetarian / Vegan Restaurant,Indian Restaurant,Jewelry Store
50,411008,ATM,Café,Park,Eastern European Restaurant,Electronics Store
85,411026,Hotel,Auto Garage,Zoo,Farmers Market,Food Truck


In [139]:
New_Pune.loc[New_Pune['Cluster Labels'] == 2, New_Pune.columns[[1] + list(range(5, New_Pune.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,411021,Farmers Market,Indian Restaurant,Flea Market,Zoo,Food Truck
42,411027,Indian Restaurant,Brewery,Vegetarian / Vegan Restaurant,Pharmacy,Snack Place
49,411045,Indian Restaurant,South Indian Restaurant,Breakfast Spot,Lounge,Chinese Restaurant
56,411021,Coffee Shop,Food Court,Indian Restaurant,Café,Shop & Service
72,411042,Diner,Indian Restaurant,Platform,Furniture / Home Store,Snack Place
125,411005,Indian Restaurant,Bar,Fast Food Restaurant,Café,Electronics Store
136,411004,Indian Restaurant,Café,Dessert Shop,Vegetarian / Vegan Restaurant,Fast Food Restaurant
149,411043,Indian Restaurant,Café,Zoo,Breakfast Spot,Food
156,411015,Indian Restaurant,Coffee Shop,Café,Zoo,Food Court
191,411028,Indian Restaurant,Coffee Shop,Pizza Place,Zoo,Food Court


In [78]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,411040,Racetrack,Track Stadium,Chinese Restaurant,Cupcake Shop,Fruit & Vegetable Store
3,411032,Deli / Bodega,Smoke Shop,Airport Lounge,Airport Service,Fried Chicken Joint
4,411035,Gym,Thai Restaurant,Coffee Shop,Café,Bakery
5,412411,Indian Restaurant,Concert Hall,Tea Room,Gym,Arts & Crafts Store
9,412411,Farmers Market,Intersection,Bus Station,English Restaurant,Department Store
12,412115,Fast Food Restaurant,Ice Cream Shop,Indian Restaurant,Gym / Fitness Center,English Restaurant
16,411046,Business Service,Women's Store,Cupcake Shop,Department Store,Dessert Shop
31,413114,Bus Station,Women's Store,Cupcake Shop,Department Store,Dessert Shop
35,411021,Indian Restaurant,Flea Market,Farmers Market,Women's Store,English Restaurant
41,411007,Indian Restaurant,Café,Ice Cream Shop,Dessert Shop,Gym


In [140]:
New_Pune.loc[New_Pune['Cluster Labels'] == 4, New_Pune.columns[[1] + list(range(5, New_Pune.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
84,411039,Café,Zoo,Fast Food Restaurant,Food Truck,Food Court
306,411024,Café,Zoo,Fast Food Restaurant,Food Truck,Food Court
470,411008,Café,Zoo,Fast Food Restaurant,Food Truck,Food Court
471,411023,Café,Zoo,Fast Food Restaurant,Food Truck,Food Court


In [95]:
zones = pd.DataFrame({"Zone":['Center', 'North', 'East', 'South', 'West'],
              
              "Latitude":[18.513021,18.578542,18.562776,18.480641,18.498793],
              "Longitude":[73.878765,73.908641,73.937058,73.862557,73.821123]})
zone.head()

,Zone,Latitude,Longitude
0,Center,18.513021,73.878765
1,North,18.578542,73.908641
2,East,18.562776,73.937058
3,South,18.480641,73.862557
4,West,18.498793,73.821123


In [100]:
Zone_venues = getNearbyVenues(names=zones['Zone'],
                                   latitudes=zones['Latitude'],
                                   longitudes=zones['Longitude'],
                                   radius = 3000
                                  )

Center
North
East
South
West


In [102]:
Zone_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Center,18.513021,73.878765,Kayani Bakery,18.514797,73.879860,Bakery
1,Center,18.513021,73.878765,Venky's Chicken,18.513503,73.879766,Fast Food Restaurant
2,Center,18.513021,73.878765,Thousand Oaks / 1000 Oaks,18.510672,73.879993,Bar
3,Center,18.513021,73.878765,JJ Garden Wada Pav,18.516492,73.878835,Food Truck
4,Center,18.513021,73.878765,Om Jai Shankar Pani Puri,18.511731,73.876414,Snack Place
5,Center,18.513021,73.878765,11 East Street Cafe,18.512403,73.880315,Bar
6,Center,18.513021,73.878765,Main Street,18.516229,73.878812,Plaza
7,Center,18.513021,73.878765,Mughal's,18.515690,73.879612,Indian Restaurant
8,Center,18.513021,73.878765,Cafe Toons,18.516272,73.878876,Sports Bar
9,Center,18.513021,73.878765,Kareem's,18.514238,73.879809,Indian Restaurant
